In [40]:
import pandas as pd
import numpy as np
import tqdm
import json

In [4]:
pd.set_option('max_columns', 100)

In [2]:
data = pd.read_csv('user_info.csv', sep=';', encoding = 'cp1251')

/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (40,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Развернем вложенные данные

In [99]:
for index, row in tqdm.tqdm(data[data.work == 1].iterrows(), total = data[data.work == 1].shape[0]):
    
    # career
    if not pd.isnull(row.career):
        
        career_str = row.career.replace('"', '').replace('\'', '"')
        
        # если работ несколько - возьмем последнюю
        split_index = career_str.rfind(', {')
        if (split_index != -1):
            career_str = career_str[:2] + career_str[split_index + 3:]
            
        try:
            career_list = json.loads(career_str)
        except json.JSONDecodeError:
            continue
        
        career_df = pd.DataFrame(career_list)
            
        # записываем все поля что есть
        for column in career_df.columns.values:
            data.loc[index, column] = career_df[column].values[0]

100%|██████████| 6553/6553 [00:34<00:00, 192.55it/s]


In [101]:
data[data.work == 1].head()

,about,activities,bdate,blacklisted,blacklisted_by_me,books,can_post,can_see_all_posts,can_see_audio,can_send_friend_request,can_write_private_message,career,city,country,crop_photo,deactivated,domain,education_form,education_status,facebook,facebook_name,faculty,faculty_name,first_name,followers_count,friend_status,games,graduation,has_mobile,has_photo,hidden,home_phone,home_town,instagram,interests,is_favorite,is_friend,is_hidden_from_feed,last_name,last_seen,livejournal,maiden_name,mobile_phone,movies,music,nickname,occupation,online,online_app,online_mobile,personal,photo_100,photo_200,photo_200_orig,photo_400_orig,photo_50,photo_id,photo_max,photo_max_orig,quotes,relation,relation_partner,relatives,schools,screen_name,sex,site,skype,status,status_audio,tv,twitter,uid,universities,university,university_name,verified,wall_comments,work,0,city_id,country_id,group_id,company,from,position,until
64,жора,NaN,21.9.2003,0.0,0.0,NaN,0,1.0,0.0,1.0,1,"[{'group_id': 149599171, 'country_id': 1, 'cit...",1463.0,1.0,"{'photo': {'pid': 456243534, 'aid': -6, 'owner...",NaN,id309828292,NaN,NaN,NaN,NaN,0.0,NaN,Саша,120.0,0,NaN,0.0,1.0,1,NaN,NaN,Зеленоград,NaN,порисульки,0.0,0.0,0.0,Сотникова,"{'time': 1510146190, 'platform': 4}",NaN,NaN,NaN,NaN,"imagine dragons, 2rbina 2rista",NaN,"{'type': 'work', 'id': 149599171, 'name': 'саш...",0,NaN,NaN,"{'langs': ['Русский', 'English'], 'religion': ...",https://pp.userapi.com/c841438/v841438388/39bb...,https://pp.userapi.com/c841438/v841438388/39bb...,https://pp.userapi.com/c841438/v841438388/39bb...,https://pp.userapi.com/c841438/v841438388/39bb...,https://pp.userapi.com/c841438/v841438388/39bb...,309828292_456243534,https://pp.userapi.com/c841438/v841438388/39bb...,https://pp.userapi.com/c841438/v841438388/39bb...,"хотелось бы верить, но верится с трудом",4.0,"{'id': 286274479, 'first_name': 'Алексей', 'la...","[{'uid': 181953537, 'type': 'sibling'}, {'uid'...","[{'id': '146100', 'country': 1, 'city': 1463, ...",id309828292,1,NaN,NaN,отнесите меня в космос,NaN,NaN,NaN,309828292,[],0.0,NaN,0.0,1.0,1.0,"[{'group_id': 149599171, 'country_id': 1, 'cit...",1.0,1.0,149599171.0,NaN,NaN,NaN,NaN
67,"Весёлый ,Умный!",Мясник!!!,10.8.1991,0.0,0.0,-,0,1.0,1.0,1.0,1,"[{'company': 'ад', 'country_id': 1, 'city_id':...",1.0,1.0,"{'photo': {'pid': 349905374, 'aid': -6, 'owner...",NaN,y.kislyonkov,NaN,NaN,NaN,NaN,0.0,NaN,Юрий,48.0,0,p b,0.0,1.0,1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Кислёнков,"{'time': 1419179635, 'platform': 7}",NaN,NaN,NaN,Самогонщики,Рок Только Рок!!!,NaN,"{'type': 'work', 'name': 'ад'}",0,NaN,NaN,"{'political': 1, 'religion': 'Православие', 'i...",https://pp.userapi.com/c625626/v625626373/ed39...,https://pp.userapi.com/c625626/v625626373/ed38...,https://pp.userapi.com/c625626/v625626373/ed36...,https://pp.userapi.com/c625626/v625626373/ed37...,https://pp.userapi.com/c625626/v625626373/ed3a...,139627373_349905374,https://pp.userapi.com/c625626/v625626373/ed38...,https://pp.userapi.com/c625626/v625626373/ed37...,NaN,1.0,NaN,"[{'uid': -500907950, 'type': 'parent', 'name':...","[{'id': '336963', 'country': 1, 'city': 16503,...",y.kislyonkov,2,NaN,NaN,NaN,NaN,не смотрю!,NaN,139627373,"[{'id': 42080, 'country': 1, 'city': 1, 'name'...",42080.0,ВГИИ (бывш. ВГАИ),0.0,1.0,1.0,"[{'company': 'ад', 'country_id': 1, 'city_id':...",42.0,1.0,NaN,ад,2008.0,Мясник,NaN
85,просто живу! какой я есть такой и буду не пыта...,авто! ЛЮБЛЮ ГОНЯТЬ НА ТАЗИКЕ!!!!!!!!!!!!!!,7.2,0.0,0.0,букварь!и всё что связана с машиной!,0,0.0,1.0,1.0,0,"[{'company': 'г наб.челны', 'country_id': 1, '...",88.0,1.0,NaN,NaN,id4676211,NaN,Доктор наук,NaN,NaN,313024.0,Высшая школа международного бизнеса,Deniska,24.0,0,варик,2015.0,0.0,1,NaN,NaN,Набережные Челны,NaN,авто! пойду гулять! да просто люблю все что ин...,0.0,0.0,0.0,Boss,"{'time': 1347726109, 'platform': 7}",NaN,NaN,NaN,это много писать И ДОЛГО НЕ ОХОТА!!!!!!!!!!!!!!!!,Noize MC !!!! Armin-van-Buuren !!!!! ...,NaN,"{'type': 'work', 'name': 'г наб.челны'}",0,NaN,NaN,"{'political': 3, 'religion

In [108]:
data.to_csv('user_info_full_career.csv', encoding = 'cp1251', sep=';', index = False)

### работа - должность

In [107]:
save = data[~pd.isnull(data.company)][~pd.isnull(data.position)][['uid', 'company', 'position']].drop_duplicates().copy()

save_cunk_size = int(save.shape[0] / 5)
for e, i in enumerate(range(0, save.shape[0] - save_cunk_size, save_cunk_size)):
    
    save[i: i + save_cunk_size].to_csv('jobs_' + str(e) + '.csv', encoding = 'cp1251', sep=';', index = False)

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
